# <center>An Analysis of Transformative Works Created by Fans of the Harry Potter Series in Reaction to the Author's Public Political Comments</center>

## <center>Project completed by Kymberlee McMaster on May 16th, 2022</center>

### <center>Introduction</center>

Fans are often known for using their talents and dedication to create new media based on the things they enjoy. One of the best examples of this is the writing and reading of fanfiction, the practice in which amateur authors may take aspects from the original content that they enjoyed and transforming them into original works of their own creation. There are various methods that these authors use to share their works with other individuals who also enjoyed the original piece of media but one of the most common is to post the work to a dedicated site for the posting and reading of fanfiction. While there are quite a few options available, we'll be focusing on Archive of Our Own, known colloquially as AO3, for our purposes as AO3's built in tagging and data storage system will allow us to search through the works of fiction using the author's own tags for their work rather than attempting to create tags ourself. 

However, since AO3 currently has over nine million works, in order to better analyze the data associated with the site and trends of fanfiction authors, we'll be focusing on writings by fans of a specific piece of media: the Harry Potter series written by J.K. Rowling.[[1]](https://archiveofourown.org/works/search?work_search%5Bquery%5D=) Additionally, we'll be specifically be focusing on the fanfiction written around a specific date in time as there are over 300,000 works for that series alone. 

On June 6th of 2020, author J.K. Rowling took to Twitter to express her displeasure over the use of the phrase "people who menstruate" rather than the word women.[[2]](https://www.glamour.com/story/a-complete-breakdown-of-the-jk-rowling-transgender-comments-controversy) This tweet and the subsequent tweets that followed it came under a lot of backlash with trans activists and fans of the Harry Potter series. This was not the first time that author J.K. Rowling had expressed such views and received backlash, but it is one of the most notable, so we will be aanalyzing works of fanfiction posted onto AO3 for the month before the tweet was made and the month following the tweet to view the potential impact that Rowling's postings may have had on the writings of the LGBTQIA+ community members and their allies. 

### <center>Data Collection</center>

As AO3 does not have a built in API, we will need to build our own method of scraping the data found on the site. In order to collect the data and avoid unneccesary scraping we'll be using AO3's built in search function to pre-search for works that were created between our dates of interest: May 6th,2020 and July 5th, 2020. We do this by accessing the Works Search page located [here](https://archiveofourown.org/works/search), and entering our parameters into the Any Search field: created_at:["2020-05-06" TO "2020-07-05"]. This will generate the link that we can use in the data scraper that will gather the information about the works for us, located [here](https://archiveofourown.org/works/search?commit=Search&page=1&work_search[bookmarks_count]=&work_search[character_names]=&work_search[comments_count]=&work_search[complete]=&work_search[creators]=&work_search[crossover]=&work_search[fandom_names]=Harry+Potter+-+J.+K.+Rowling&work_search[freeform_names]=&work_search[hits]=&work_search[kudos_count]=&work_search[language_id]=&work_search[query]=created_at%3A[%222020-05-06%22+TO+%222020-07-05%22]&work_search[rating_ids]=&work_search[relationship_names]=&work_search[revised_at]=&work_search[single_chapter]=0&work_search[sort_column]=created_at&work_search[sort_direction]=asc&work_search[title]=&work_search[word_count]=). 

By looking at our search results, we will see that we will be scraping the information about 8,784 works that were created and area available publicly without an account in the 2 month time period we've identified. Below, we will first import the libararies necessary for this project. 

In [25]:
# Import the libraries necessary to complete this project 
import requests
import math
from bs4 import BeautifulSoup 
import csv
import pandas as pd 
import numpy as np 
from datetime import datetime 


import json 
import os.path 
import matplotlib.pyplot as plt 

Next, we need to inspect the page to see how the data on each page is stored. By using the developers tools, we can see that the results of the search are displayed in a class identified as a “works index group" and each work is a list item below that with the role "article". We know that there are 8,784 works to be consumed, and there are 20 works displayed on each search page so we'll need to request the informtion from 440 pages. We split the URL into parts before and after the page number is stored to that we can complete our requests through an iterative process which automatically updates the page number used to request data. Then we initialize a couple new files to store the content from the scraping as we'll be completing it in page portions and that would make it difficult to store as a pandas dataframe right off the bat. 

In [30]:
#Split the URL into parts and store the current page number with those parts 
urlpt1 = "https://archiveofourown.org/works/search?commit=Search&page="
currpagenum = 1
urlpt2 = "&work_search[bookmarks_count]=&work_search[character_names]=&work_search[comments_count]=&work_search[complete]=&work_search[creators]=&work_search[crossover]=&work_search[fandom_names]=Harry+Potter+-+J.+K.+Rowling&work_search[freeform_names]=&work_search[hits]=&work_search[kudos_count]=&work_search[language_id]=&work_search[query]=created_at%3A[%222020-05-06%22+TO+%222020-07-05%22]&work_search[rating_ids]=&work_search[relationship_names]=&work_search[revised_at]=&work_search[single_chapter]=0&work_search[sort_column]=created_at&work_search[sort_direction]=asc&work_search[title]=&work_search[word_count]="

#Identified the number of works and pages that the scraper will need to iterate through
works = 8784
pages = math.ceil(works/20)

#Iniate a new file to store the basic content from the scraping 
header = ['Title', 'Author', 'ID', 'Date_updated', 'Rating', 'Pairing', 'Warning', 'Complete', 'Language', 'Word_count', 'Num_chapters', 'Num_comments', 'Num_kudos', 'Num_bookmarks', 'Num_hits']
with open('storedbasic.csv','w', encoding='utf8') as storedbasic:
    writer = csv.writer(storedbasic)
    writer.writerow(header)

#Initiate a new file to store the more complex text content from the scraping
header_row = ['ID', 'Tags', 'Summary', 'Date_published']
with open('storedcomplex.csv','w', encoding='utf8') as storedcomplex:
    writer = csv.writer(storedcomplex)
    writer.writerow(header_row)

Now that we've completed some of our basic work, we can begin to design some of the functions we'll need to call to scrape the data out of the page. Specifically, we'll need one function to read in the basic content and another to read in the more complex text content such as the tags. One piece of information that we could scrape but are choosing not to for the purpose of this project is the Comments on the published works. While this could have some interested information for us to take a look at, we are already dealing with an extremely large amount of data and the Comments section of each work does not have any bearing on what our true aim is with this project. 

In [31]:
#Function to gather basic data 

#Function to gather complex text data 

With our helper functions defined, we can now define our main function to iterate through the pages of the searched works and gather the data into the CSV files previously created. 

In [ ]:
page = requests.get(urlpt1 + str(currpagenum) + urlpt2)

#Use BeautifulSoup to parse the data as html
soup = BeautifulSoup(page.content, "html.parser")

Our data has been stored into CSV files and we can now transform our data into pandas data frames. First we will create a pandas data frame that stores the basic data and then another that stores the complex text data. Then we Join both data frames to create one master dataframe that we can use to view and do any tidying that may be necessary. 